# This is a PyTorch Tutorial from PyTorch Official Site
link: [quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch offers domain-specific libraries such as 

1. [TorchText](https://pytorch.org/text/stable/index.html), 
2. [TorchVision](https://pytorch.org/text/stable/index.html), and 
3. [TorchAudio](https://pytorch.org/text/stable/index.html)

all of which include datasets. For this tutorial, we will be using a TorchVision dataset.

The `torchvision.datasets` module contains `Dataset` objects for many real-world vision data like **CIFAR**, **COCO** ([full list here](https://pytorch.org/text/stable/index.html)). In this tutorial, we use the `FashionMNIST` dataset. Every TorchVision `Dataset` includes two arguments: `transform` and `target_transform` to modify the samples and labels respectively.

In [3]:
# Download training data from dataset
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from dataset
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the `Dataset` as an argument to `DataLoader`. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a **batch size of 64**, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [11]:
batch_size = 64

# Create data loader
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(training_data, batch_size=batch_size)

for X, y in test_dataloader:
    print('Shape of X [N, C, H, W]:', X.shape)
    print('Shape of y:', y.shape, y.dtype)
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Creating Models
To define a neural network in PyTorch, we create a class that inherits from [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). We define the layers of the network in the `__init__` function and specify how data will pass through the network in the `forward` function. To accelerate operations in the neural network, we move it to the GPU if available.

In [13]:
# Get CPU or GPU device for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using {} device'.format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## Optimizing the Model Parameters
To train model, we need [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](https://pytorch.org/docs/stable/optim.html).

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

We also check the model’s performance against the test dataset to ensure it is learning.

In [17]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg. Loss: {test_loss:>8f} \n')

The training process is conducted over several iterations (**epochs**). During each **epoch**, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [18]:
epoch = 5

for t in range(epoch):
    print(f'Epoch {t+1}\n------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(train_dataloader, model, loss_fn)
print('Done')

Epoch 1
------------------------------
loss: 2.306065 [    0/60000]
loss: 2.295849 [ 6400/60000]
loss: 2.292001 [12800/60000]
loss: 2.288401 [19200/60000]
loss: 2.268022 [25600/60000]
loss: 2.265650 [32000/60000]
loss: 2.267147 [38400/60000]
loss: 2.247698 [44800/60000]
loss: 2.256712 [51200/60000]
loss: 2.233366 [57600/60000]
Test error: 
 Accuracy: 49.2%, Avg. Loss: 2.224048 

Epoch 2
------------------------------
loss: 2.243088 [    0/60000]
loss: 2.227201 [ 6400/60000]
loss: 2.204577 [12800/60000]
loss: 2.212039 [19200/60000]
loss: 2.153504 [25600/60000]
loss: 2.156748 [32000/60000]
loss: 2.174996 [38400/60000]
loss: 2.129626 [44800/60000]
loss: 2.164420 [51200/60000]
loss: 2.105088 [57600/60000]
Test error: 
 Accuracy: 50.0%, Avg. Loss: 2.087146 

Epoch 3
------------------------------
loss: 2.137674 [    0/60000]
loss: 2.106386 [ 6400/60000]
loss: 2.046973 [12800/60000]
loss: 2.070414 [19200/60000]
loss: 1.934612 [25600/60000]
loss: 1.946007 [32000/60000]
loss: 2.000894 [38400/6

# Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [19]:
torch.save(model.state_dict(), 'model.pth')
print('Saved PyTorch model into model.pth')

Saved PyTorch model into model.pth


## Loading Model
The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [20]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

This model can now be used to make predictions.

In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
